In [6]:
import scipy
matlab = scipy.io.loadmat('Xtrain.mat')

In [7]:
X_train = matlab['Xtrain']

In [8]:
import plotly.express as px
import plotly.graph_objects as go

In [9]:
# Convert the data to a list
X_train_list = X_train.flatten().tolist()
print(X_train_list)

# Plot a line of the values in the training set
fig = px.line(y=X_train_list, x=range(len(X_train_list)), title='Line Plot of X_train Values')
fig.update_layout(xaxis_title='Index', yaxis_title='Value') 
fig.show()

[86, 141, 95, 41, 22, 21, 32, 72, 138, 111, 48, 23, 19, 27, 59, 129, 129, 58, 27, 19, 24, 46, 112, 144, 73, 30, 20, 19, 37, 92, 152, 93, 36, 20, 18, 29, 71, 146, 117, 46, 23, 18, 22, 52, 128, 142, 62, 26, 17, 19, 37, 100, 158, 86, 32, 17, 17, 27, 72, 154, 118, 43, 20, 15, 21, 47, 128, 150, 63, 24, 16, 17, 33, 92, 166, 95, 33, 16, 14, 22, 58, 149, 137, 50, 20, 15, 16, 35, 106, 169, 82, 28, 15, 14, 23, 65, 160, 133, 45, 18, 14, 16, 36, 111, 176, 80, 26, 14, 13, 20, 60, 163, 140, 45, 18, 12, 14, 30, 98, 185, 93, 28, 14, 12, 17, 46, 145, 167, 58, 19, 11, 12, 21, 68, 183, 132, 38, 15, 11, 13, 26, 93, 202, 100, 27, 12, 10, 13, 30, 114, 206, 82, 21, 11, 9, 10, 30, 120, 215, 77, 20, 10, 8, 9, 21, 95, 234, 99, 22, 9, 8, 7, 9, 35, 176, 215, 46, 11, 7, 5, 4, 3, 4, 19, 105, 125, 55, 30, 28, 41, 71, 98, 79, 49, 34, 33, 47, 75, 95, 76, 48, 34, 35, 49, 78, 96, 75, 47, 34, 34, 49, 81, 99, 76, 46, 32, 33, 49, 83, 101, 74, 44, 32, 33, 50, 85, 103, 73, 43, 30, 31, 49, 85, 106, 74, 43, 31, 31, 49, 87, 109

In [10]:
import pandas as pd

In [11]:
def create_features(timeseries_array, n_lags):
    df = pd.DataFrame(timeseries_array, columns=['label'])
    for i in range(1, n_lags + 1):
        df[f'lag {i}'] = df['label'].shift(i)
    df = df.dropna()

    X = df.drop(columns=['label']).to_numpy()
    y = df['label'].to_numpy()

    return X, y

X, y = create_features(X_train_list, 10)

print(X.shape)
print(y.shape)

(990, 10)
(990,)


In [12]:
def expanding_window_cv_sets(X, y, folds: int, validation_split_ratio: float = 0.2):
    
    n = len(X)
    fold_size = int(n / folds)

    for fold in range(folds):
        start_fold = fold * fold_size
        end_fold = start_fold + fold_size

        training_start = 0
        training_end = int(end_fold - fold_size * validation_split_ratio)

        validation_start = training_end

        training_set = X[training_start:training_end], y[training_start:training_end]
        validation_set = X[validation_start:end_fold], y[validation_start:end_fold]
        yield training_set, validation_set

# Example usage
folds = 5
for train_set, val_set in expanding_window_cv_sets(X, y, folds):
    train_X, train_y = train_set
    val_X, val_y = val_set

    print("Training Set:")
    print(train_X.shape, train_y.shape)
    print("Validation Set:")
    print(val_X.shape, val_y.shape)
    print("\n")


Training Set:
(158, 10) (158,)
Validation Set:
(40, 10) (40,)


Training Set:
(356, 10) (356,)
Validation Set:
(40, 10) (40,)


Training Set:
(554, 10) (554,)
Validation Set:
(40, 10) (40,)


Training Set:
(752, 10) (752,)
Validation Set:
(40, 10) (40,)


Training Set:
(950, 10) (950,)
Validation Set:
(40, 10) (40,)




In [24]:
# Create two tables to store the results of CV grid search
epoch_grid_search_results = pd.DataFrame(columns=['epoch', 'fold', 'MSE', 'MAE'])
grid_search_results = pd.DataFrame(columns=['hidden_units', 'lags', 'fold', 'MSE', 'MAE'])

In [25]:
FOLDS = 5
EPOCHS = [10, 20, 50, 100, 500, 1000]
LAGS = [5, 15, 25, 35, 50]
HIDDEN_UNITS = [5, 10, 20, 50, 100]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

X, y = create_features(X_train_list, 25)

for epoch in EPOCHS:
    print("Epoch: ", epoch)
    for fold, (train_set, val_set) in enumerate(expanding_window_cv_sets(X, y, folds)):

        train_X, train_y = train_set
        val_X, val_y = val_set
        
        # Reshape the data to be 3D for LSTM input
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))

        # First, we are gonna fix the LSTM to 10 hidden units and grid search the epochs

        # Define the lSTM model with 15 hidden units
        model = Sequential([
            LSTM(10, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mse')
        model.fit(train_X, train_y, epochs=epoch, batch_size=32, verbose=0)

        # Recursively predict the validation set, so start with the first example of the validation set and use the model to predict the next value, then use that value to predict the next one, and so on.
        current_input = val_X[0].reshape((1, 1, val_X.shape[2]))
        y_pred = []

        for i in range(len(val_y)):
            new_y = model.predict(current_input)
            y_pred.append(new_y[0][0])
            # Remove the first value of the current input and append the new value to the end of the input sequence
            current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)
        
        y_pred = np.array(y_pred).reshape(-1, 1)
        print("Epochs: ", epoch)
        print("Standardized MSE: ", mean_squared_error(val_y, y_pred))
        print("Standardized MAE: ", np.mean(np.abs(val_y - y_pred)))
        print("\n")
        
        # Append the results to the epoch grid search results table
        epoch_grid_search_results = pd.concat([epoch_grid_search_results, pd.DataFrame({'fold': [fold],'epoch': [epoch], 'MSE': [mean_squared_error(val_y, y_pred)], 'MAE': [np.mean(np.abs(val_y - y_pred))]})], ignore_index=True)

In [ ]:
display(epoch_grid_search_results)
epoch_grid_search_results.to_csv('epoch_grid_search_results.csv', index=False)


In [ ]:
import pandas as pd
epoch_grid_search_results = pd.read_csv('epoch_grid_search_results.csv')

In [ ]:
epoch_grid_search_results.groupby('epoch').agg({'MSE': 'mean', 'MAE': 'mean'})

In [ ]:
BEST_EPOCH = epoch_grid_search_results.loc[epoch_grid_search_results['MSE'].idxmin()]['epoch']
print("Best epoch: ", BEST_EPOCH)

In [ ]:

for lags in LAGS:
    print(f"Lag: {lags}")
    X, y = create_features(X_train_list, lags)

    # Expanding window cross-validation
    for fold, (train_set, validation_set) in enumerate(expanding_window_cv_sets(X, y, FOLDS)):
        train_X, train_y = train_set
        val_X, val_y = validation_set

        # Reshape the data to be 3D for LSTM input
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))

        for hidden_units in HIDDEN_UNITS:
            # Define the LSTM model with the current number of hidden units
            model = Sequential([
                LSTM(hidden_units, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False),
                Dense(1)
            ])

            model.compile(optimizer='adam', loss='mse')
            model.fit(train_X, train_y, epochs=BEST_EPOCH, batch_size=32, verbose=0)

            # Recursively predict the validation set
            current_input = val_X[0].reshape((1, 1, val_X.shape[2]))
            y_pred = []

            for i in range(len(val_y)):
                new_y = model.predict(current_input)
                y_pred.append(new_y[0][0])
                current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)

            y_pred = np.array(y_pred).reshape(-1, 1)
            print(f"Lag: {lags}, Hidden Units: {hidden_units}")
            print("Standardized MSE: ", mean_squared_error(val_y, y_pred))
            print("Standardized MAE: ", np.mean(np.abs(val_y - y_pred)))
            print("\n")

            # Append the results to the lag grid search results table
            grid_search_results = pd.concat([grid_search_results, pd.DataFrame({'lag': [lags], 'hidden_units': [hidden_units], 'MSE': [mean_squared_error(val_y, y_pred)], 'MAE': [np.mean(np.abs(val_y - y_pred))]})], ignore_index=True)

In [ ]:
display(grid_search_results)
grid_search_results.to_csv('grid_search_results.csv', index=False)

In [13]:
matlab2 = scipy.io.loadmat('Xtest.mat')
X_matlab_test = matlab2['Xtest']
X_test_list = X_matlab_test.flatten().tolist()

concat_train_test = X_train_list + X_test_list
fig = px.line(y=concat_train_test, x=range(len(concat_train_test)), title='Line Plot of X_train and X_test Values')
fig.update_layout(xaxis_title='Index', yaxis_title='Value')
fig.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

X, y = create_features(X_train_list, 35)
X = X.reshape((X.shape[0], 1, X.shape[1]))
y = y.reshape((y.shape[0], 1))

model = Sequential([
    LSTM(11, activation='relu', input_shape=(X.shape[1], X.shape[2]), dropout=0.2, return_sequences=True),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=400, batch_size=32, verbose=0)

# Now test the model on the test set, first create the features for the test set
X_test, y_test = create_features(X_test_list, 35)
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
y_test = y_test.reshape((y_test.shape[0], 1))

# Recursively predict the test set
current_input = X_test[0].reshape((1, 1, X_test.shape[2]))
y_test_pred = []

for i in range(len(y_test)):
    new_y = model.predict(current_input)
    y_test_pred.append(new_y[0][0])
    current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)

y_test_pred = np.array(y_test_pred).reshape(-1, 1)

# Plot the predictions vs actual values
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test.flatten(), mode='lines', name='Actual Values'))
fig.add_trace(go.Scatter(y=y_test_pred.flatten(), mode='lines', name='Predicted Values'))
fig.update_layout(title='LSTM Predictions vs Actual Values', xaxis_title='Index', yaxis_title='Value')
fig.show()

C:\Users\niek0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━